In [ ]:
import pandas as pd
import numpy as np
import tempfile

In [2]:
import ROOT
# if the ROOT import fails, you can still run other sections that do not use it

In [31]:
N = 100000

### Pandas

In [ ]:
df = pd.DataFrame({
    "col1": np.random.random(N),
    "col2": np.random.random(N),
    })

# different methods of accessing data in Pandas:
# vectorization
%timeit np.sqrt(df["col1"]**2 + df["col2"]**2)

# itertuples 
def h():
    for tup in df.itertuples():
        np.sqrt(tup[0]**2 + tup[1]**2)
%timeit h()

# access by index
def f():
    for i in range(len(df)):
        np.sqrt(df.iloc[i]["col1"]**2 + df.iloc[i]["col2"]**2)
%timeit f()

# iterrows
def g():
    for i,row in df.iterrows():
        np.sqrt(row["col1"]**2 + row["col2"]**2)
%timeit g()

468 μs ± 65.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
108 ms ± 665 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.75 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.64 s ± 402 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ROOT

In [33]:
# ROOT
from ROOT import TTree, TFile

with tempfile.TemporaryDirectory() as tempdir:

    file = TFile.Open(f"{tempdir}/DataTree.root", "RECREATE")
    tree = TTree(f"DataTree", "DataTree")
    col1_val = np.array([0.],dtype=np.float64)
    col2_val = np.array([0.],dtype=np.float64)
    tree.Branch("col1", col1_val, f'col1/D')
    tree.Branch("col2", col2_val, f'col2/D') 

    # Fill the tree
    for i in range(N):
        col1_val = np.array([np.random.random()],dtype=np.float64)
        col2_val = np.array([np.random.random()],dtype=np.float64)
        tree.Fill()

    tree.Write()
    file.Close()

    file2 = TFile.Open(f"{tempdir}/DataTree.root")
    tree2 = file2.Get("DataTree")
    
    def h():
        for x in tree2:
            np.sqrt(x.col1**2 + x.col2**2)

    %timeit h()
    file2.Close()

458 ms ± 163 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
